In [1345]:
import pandas as pd
import math
import numpy as np

In [1346]:
def convert_and_extract(value):
    if pd.notna(value):
        value = value.replace(',', '.')[:8]
        return value
    else:
        return np.nan


In [1347]:
df_clientes = pd.read_parquet('/home/pydev/workflow/dt_unifed_statistics/data/clientes_b2b.parquet')
df_clientes["LONGITUD"] = df_clientes['LONGITUD'].apply(convert_and_extract)
df_clientes["LATITUD"]  = df_clientes['LATITUD'].apply(convert_and_extract)
df_clientes.head(1)
# df_clientes.info()

,COD_CIA,ROWID_TERCERO,NIT_TERCERO,ID_TERCERO,RAZON_SOCIAL,COD_SUCURSAL_CLIENTE,DES_SUCURSAL_CLIENTE,IND_ESTADO_ACTIVO_CLIENTE,COD_DIA_VISITA,DES_DIA_VISITA,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,COD_CANAL,LONGITUD,LATITUD
0,2,41367.0,None,None,PUNTO DE VENTA CLIENTE OCASIONAL,150,CALL CENTER,1,None,None,None,None,STORE,-75.5693,6.22577


In [1348]:
df_visitas = pd.read_parquet('/home/pydev/workflow/dt_unifed_statistics/data/visitas_vendedores.parquet')
df_visitas['FECHAS_VISITA'] = pd.to_datetime(df_visitas['Inicio_Visita'])
df_visitas["Logintud"]      = df_visitas['Logintud'].apply(convert_and_extract)
df_visitas["Latitud"]       = df_visitas['Latitud'].apply(convert_and_extract)
df_visitas['AÑO']           = df_visitas['FECHAS_VISITA'].dt.year.astype(str)
df_visitas                  = df_visitas[df_visitas["AÑO"]=='2023']
df_visitas['FECHAS_VISITA'] = df_visitas['FECHAS_VISITA'].dt.date.astype(str)
df_visitas.head(1)

,Logintud,Latitud,Inicio_Visita,Finalizacion_Visita,Razon_Social_Cliente,ID_TERCERO,Nit,Codigo_Sucursal,Codigo_Vendedor,Des_vendedor,FECHAS_VISITA,AÑO
109,-74.1781,4.764811,2023-01-04 08:33:43,NaT,CARLOS ERNESTO HERRRA ALFONSO,1070597717-001,1070597717,001,0332,VEND CACERES JUAN CARLOS BUSTOS,2023-01-04,2023


In [1349]:
df_visitas.shape

(10922, 12)

In [1350]:
# df_visitas_temp = df_visitas[df_visitas["ID_TERCERO"]=="1005566146-001"]
# df_visitas_temp

In [1351]:
df_combined = pd.merge(df_clientes, df_visitas)
df_combined.head(1)

,COD_CIA,ROWID_TERCERO,NIT_TERCERO,ID_TERCERO,RAZON_SOCIAL,COD_SUCURSAL_CLIENTE,DES_SUCURSAL_CLIENTE,IND_ESTADO_ACTIVO_CLIENTE,COD_DIA_VISITA,DES_DIA_VISITA,...,Latitud,Inicio_Visita,Finalizacion_Visita,Razon_Social_Cliente,Nit,Codigo_Sucursal,Codigo_Vendedor,Des_vendedor,FECHAS_VISITA,AÑO
0,2,43525.0,51603652,51603652-001,DELGADO PRADO RUTH ESTELA,001,DISTRICOSTA,1,004,Miercoles,...,4.616238,2023-07-05 08:56:04,NaT,RUTH ESTELA DELGADO PRADO,51603652,001,0312,VEND DIAZ YULI JOHANNA ALDANA,2023-07-05,2023


In [1352]:
# Extract the info of clients by this columns:
df_combined_1 = df_combined.copy()
df_combined_1 = df_combined_1[['ID_TERCERO', 'COD_DIA_VISITA','DES_DIA_VISITA','COD_FRECUENCIA_VISITA','DES_FRECUENCIA_VISITA']]
df_combined_1.drop_duplicates(subset=["ID_TERCERO"], inplace=True)
# df_combined_1["type"] = "clientes_copy_1"
df_combined_1.head(3)

,ID_TERCERO,COD_DIA_VISITA,DES_DIA_VISITA,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA
0,51603652-001,004,Miercoles,003,Quincena 2 y 4
1,71778267-003,002,Lunes,003,Quincena 2 y 4
2,73556392-001,002,Lunes,001,Semanal


In [1353]:
df_calendario = pd.read_parquet('/home/pydev/workflow/dt_unifed_statistics/data/calendario_visitas.parquet')
df_calendario = df_calendario.iloc[:, [0, 1, 3, 5, 7]]
# Usar el método .melt() para "desapilar" las columnas de códigos
df_calendario = pd.melt(df_calendario, id_vars=["FECHA", "COD_DIA VISITA"], value_vars=["COD SEMANAL",	"COD QUINCENAL","COD MENSUAL"], var_name="Tipo_Codigo", value_name="Codigo_Frecuencia")
# Cambiar el nombre de la columna utilizando el método .rename()
df_calendario.rename(columns={'COD_DIA VISITA': 'COD_DIA_VISITA','Codigo_Frecuencia':'COD_FRECUENCIA_VISITA', 'FECHA': 'FECHAS_VISITA', 'Tipo_Codigo':'TIPO_CODIGO'}, inplace=True)
df_calendario['FECHAS_VISITA'] = pd.to_datetime(df_calendario['FECHAS_VISITA'], format= "%d/%m/%Y").astype(str)
# df_calendario["type_2"] = "calendario_visitas"
df_calendario = df_calendario.iloc[:, [0, 1, 3]]
df_calendario.head(3)
# df_calendario.info()

,FECHAS_VISITA,COD_DIA_VISITA,COD_FRECUENCIA_VISITA
0,2023-01-01,001,001
1,2023-01-02,002,001
2,2023-01-03,003,001


In [1354]:
# Agrega una nueva columna "Cumple" al DataFrame de visitas realizadas
df_combined_2 = df_combined.copy()
df_combined_2 = df_combined_2[['ID_TERCERO', 'COD_DIA_VISITA','FECHAS_VISITA','DES_DIA_VISITA','COD_FRECUENCIA_VISITA','DES_FRECUENCIA_VISITA']]
df_combined_2['Cumple'] = False  # Inicialmente, asumimos que no se cumple en ninguna fecha

# Itera a través del DataFrame de visitas realizadas y marca las fechas que se cumplen
for index, row in df_combined_2.iterrows():
    fecha_realizada = row['FECHAS_VISITA']
    codigo_dia = row['COD_DIA_VISITA']
    codigo_frec = row['COD_FRECUENCIA_VISITA']

    calendario_filtrado = df_calendario[(df_calendario['COD_DIA_VISITA'] == codigo_dia) &
                                              (df_calendario['COD_FRECUENCIA_VISITA'] == codigo_frec)]

    if fecha_realizada in calendario_filtrado['FECHAS_VISITA'].values:
        df_combined_2.at[index, 'Cumple'] = True

# Mostrar el DataFrame de visitas realizadas con la columna "Cumple"
df_combined_2

,ID_TERCERO,COD_DIA_VISITA,FECHAS_VISITA,DES_DIA_VISITA,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,Cumple
0,51603652-001,004,2023-07-05,Miercoles,003,Quincena 2 y 4,True
1,71778267-003,002,2023-05-23,Lunes,003,Quincena 2 y 4,False
2,73556392-001,002,2023-04-03,Lunes,001,Semanal,True
3,7919980-001,003,2023-05-02,Martes,001,Semanal,True
4,800105842-001,005,2023-07-18,Jueves,001,Semanal,False
...,...,...,...,...,...,...,...
10827,1053332351-001,000,2023-09-13,No determinado,000,No determinado,False
10828,901549752-001,005,2023-09-11,Jueves,002,Quincena 1 y 3,False
10829,901549752-001,005,2023-09-11,Jueves,002,Quincena 1 y 3,False
10830,901549752-001,005,2023-09-11,Jueves,002,Quincena 1 y 3,False


In [1355]:
# Cross Join 
df_clientes_visitas = pd.merge(df_combined_1, df_calendario)
df_clientes_visitas.head(3)
# df_clientes_visitas.shape

,ID_TERCERO,COD_DIA_VISITA,DES_DIA_VISITA,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,FECHAS_VISITA
0,51603652-001,004,Miercoles,003,Quincena 2 y 4,2023-01-04
1,51603652-001,004,Miercoles,003,Quincena 2 y 4,2023-01-18
2,51603652-001,004,Miercoles,003,Quincena 2 y 4,2023-02-01


In [1356]:
# df_combined.info()

In [1357]:
df_base = df_combined.iloc[:, [3, 8,9,10,11, 13, 14, 15, 16, 17, 18, 22, 24]]
df_base.head(3)
# df_base.shape

,ID_TERCERO,COD_DIA_VISITA,DES_DIA_VISITA,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,LONGITUD,LATITUD,Logintud,Latitud,Inicio_Visita,Finalizacion_Visita,Codigo_Vendedor,FECHAS_VISITA
0,51603652-001,004,Miercoles,003,Quincena 2 y 4,-74.0836,4.606358,-74.1672,4.616238,2023-07-05 08:56:04,NaT,0312,2023-07-05
1,71778267-003,002,Lunes,003,Quincena 2 y 4,-75.1799,6.224823,-75.1792,6.221319,2023-05-23 09:40:27,NaT,0123,2023-05-23
2,73556392-001,002,Lunes,001,Semanal,-75.5513,10.42415,-75.5051,10.38248,2023-04-03 14:32:04,2023-04-03 14:32:54,0502,2023-04-03


In [1358]:
df_result               = pd.merge(df_clientes_visitas, df_base, how="outer")
df_result               = df_result.sort_values(by=['ID_TERCERO', 'Codigo_Vendedor'], ascending=[True, False])
df_result['LONGITUD']   = pd.to_numeric(df_result['LONGITUD'], errors='coerce')
df_result['LATITUD']    = pd.to_numeric(df_result['LATITUD'], errors='coerce')
df_result['Logintud']   = pd.to_numeric(df_result['Logintud'], errors='coerce')
df_result['Latitud']    = pd.to_numeric(df_result['Latitud'], errors='coerce')
# df_result               = df_result[df_result['ID_TERCERO']=='1000007753-001']
# df_result.info()

In [1359]:
# Convert the datetime columns to datetime objects
df_result['StartDatetime'] = pd.to_datetime(df_result['Inicio_Visita'], format='%m/%d/%Y %H:%M')
df_result['EndDatetime'] = pd.to_datetime(df_result['Finalizacion_Visita'], format='%m/%d/%Y %H:%M')
# df_result['Inicio_Visita_F'] = df_result['Inicio_Visita'].dt.date
# Calculate the time difference
df_result['TimeDifference'] = df_result['EndDatetime'] - df_result['StartDatetime']
# Extract hours and minutes and format as a single column
df_result['TimeDifference'] = df_result['TimeDifference'].apply(lambda x: f"{x.seconds//3600}h:{x.seconds%3600//60} min")
df_result.drop(['StartDatetime','EndDatetime'],axis=1, inplace=True)
df_result['TimeDifference'] = ['' if x == 'nanh:nan min' else x for x in df_result['TimeDifference']]

In [1360]:
# Fórmula Haversine
def haversine(lon1, lat1, lon2, lat2):
 # Radio de la Tierra en kilómetros
  R = 6371.0
   # Convierte las coordenadas de grados a radianes
  lat1 = math.radians(lat1)
  lon1 = math.radians(lon1)
  lat2 = math.radians(lat2)
  lon2 = math.radians(lon2)
  # Diferencias de latitud y longitud
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  #Se usa la ecuación "a = [sin ² (dlat / 2) + cos (lat1)] x cos (lat2) x sin ² (dlong / 2)"
  a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  #Distancia en metros
  distance = R * c *1000
  return distance

In [1361]:
#Arreglo para almacenar la distancia entre los puntos
DistanciasHaversine =[]
# Recorre las filas del DataFrame
for indice, fila in df_result.iterrows():
  lon1 = fila['Logintud']
  lat1 = fila['Latitud']
  lon2 = fila['LONGITUD']
  lat2 = fila['LATITUD']
  #print(fila)
  if(lon1 != 0 and lon2 != 0 and lat1 != 0 and lat2 != 0):
    total_H = haversine(lon1, lat1, lon2, lat2)
    DistanciasHaversine.append(total_H)
  else:
    DistanciasHaversine.append(0)

df_result["Distancia_Haversine"] = DistanciasHaversine



In [1362]:
# df_result.info()

In [1363]:
# df_result['Distancia_Haversine'] = df_result['Distancia_Haversine'].fillna(0)
df_result['Rango_Visita'] = ['Dentro del Rango' if score <= 200 else ('Fuera del Rango' if score >= 200 else '') for score in df_result['Distancia_Haversine']]
df_result['Cumpl_Rutero'] = ['Si' if score <= 200 else ('Fuera del Rango' if score >= 200 else '') for score in df_result['Distancia_Haversine']]

In [1364]:
df_calendario.shape

(1095, 3)

In [1365]:
# df_base['date_column']     = pd.to_datetime(df_result['FECHAS_VISITA'])
# df_calendario['date_column'] = pd.to_datetime(df_calendario['FECHAS_VISITA'])

In [1366]:
# df_base['comparison_result'] = df_base['date_column'].isin(df_calendario['date_column']).astype(int)


In [1367]:
# df_base.to_csv('base.csv')

In [1368]:
# merged_df

In [1369]:
# # Create an empty list to store the comparison results
# comparison_results = []

# # Iterate through the rows of df1
# for index, row in df_result.iterrows():
#     date_value = row['date_column']
#     # Check if the date exists in df2
#     if date_value in df_calendario['date_column'].values:
#         comparison_results.append(1)
#     else:
#         comparison_results.append(0)

# # Add the comparison results as a new column in df1
# df_result['comparison_result'] = comparison_results

In [1370]:
df_combined_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10832 entries, 0 to 10831
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID_TERCERO             10832 non-null  object
 1   COD_DIA_VISITA         10832 non-null  object
 2   FECHAS_VISITA          10832 non-null  object
 3   DES_DIA_VISITA         10832 non-null  object
 4   COD_FRECUENCIA_VISITA  10832 non-null  object
 5   DES_FRECUENCIA_VISITA  10832 non-null  object
 6   Cumple                 10832 non-null  bool  
dtypes: bool(1), object(6)
memory usage: 861.0+ KB


In [1371]:
df_combined_2 = df_combined_2.iloc[:, [0,2,6]]
# df_combined_2 = df_combined_2[df_combined_2["Cumple"]==True]
# df_combined_2.drop_duplicates(subset=['FECHAS_VISITA'])
# df_combined_2.to_csv('temp3.csv')
df_combined_2.head()

,ID_TERCERO,FECHAS_VISITA,Cumple
0,51603652-001,2023-07-05,True
1,71778267-003,2023-05-23,False
2,73556392-001,2023-04-03,True
3,7919980-001,2023-05-02,True
4,800105842-001,2023-07-18,False


In [1372]:
merged_df_final_1 = pd.merge( df_result, df_combined_2)
# merged_df_final_1.drop_duplicates()
merged_df_final_1.to_csv('temp2.csv')

In [1373]:
print(merged_df_final_1.shape)
print(df_result.shape)
# 71610022-001
# 900561159-003


(21364, 18)
(96557, 17)


In [1374]:
df_result.to_csv('temp.csv')

In [1375]:
df_result.head()

,ID_TERCERO,COD_DIA_VISITA,DES_DIA_VISITA,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,FECHAS_VISITA,LONGITUD,LATITUD,Logintud,Latitud,Inicio_Visita,Finalizacion_Visita,Codigo_Vendedor,TimeDifference,Distancia_Haversine,Rango_Visita,Cumpl_Rutero
94410,1000007753-001,002,Lunes,004,Mensual Semana 1,2023-05-03,-74.2119,4.717797,-74.2119,4.717885,2023-05-03 11:34:42,2023-05-03 11:38:27,0304,0h:3 min,9.785154,Dentro del Rango,Si
94411,1000007753-001,002,Lunes,004,Mensual Semana 1,2023-05-24,-74.2119,4.717797,-74.2117,4.717827,2023-05-24 11:07:54,2023-05-24 11:11:22,0304,0h:3 min,22.413270,Dentro del Rango,Si
94412,1000007753-001,002,Lunes,004,Mensual Semana 1,2023-06-05,-74.2119,4.717797,-74.2119,4.717946,2023-06-05 10:18:44,2023-06-05 10:24:08,0304,0h:5 min,16.568044,Dentro del Rango,Si
94413,1000007753-001,002,Lunes,004,Mensual Semana 1,2023-06-26,-74.2119,4.717797,-74.2119,4.717843,2023-06-26 13:05:54,2023-06-26 13:09:14,0304,0h:3 min,5.114967,Dentro del Rango,Si
94414,1000007753-001,002,Lunes,004,Mensual Semana 1,2023-07-04,-74.2119,4.717797,-74.2302,4.706338,2023-07-04 15:44:27,2023-07-04 16:33:40,0304,0h:49 min,2395.053853,Fuera del Rango,Fuera del Rango


In [1376]:
# df_combined.info()